In [1]:
import tensorflow as tf
import numpy as np

tf.__version__

'2.0.0'

In [2]:
def spectral_norm(w, iteration=1):
    w_shape = w.shape.as_list()
    w = tf.reshape(w, [-1, w_shape[-1]])

    u = tf.compat.v1.get_variable("u", [1, w_shape[-1]], initializer=tf.random_normal_initializer(), trainable=False)

    u_hat = u
    v_hat = None
    for i in range(iteration):
        """
        power iteration
        Usually iteration = 1 will be enough
        """
        v_ = tf.matmul(u_hat, tf.transpose(w))
        v_hat = tf.nn.l2_normalize(v_)

        u_ = tf.matmul(v_hat, w)
        u_hat = tf.nn.l2_normalize(u_)

    u_hat = tf.stop_gradient(u_hat)
    v_hat = tf.stop_gradient(v_hat)

    sigma = tf.matmul(tf.matmul(v_hat, w), tf.transpose(u_hat))

    with tf.control_dependencies([u.assign(u_hat)]):
        w_norm = w / sigma
        w_norm = tf.reshape(w_norm, w_shape)


    return w_norm

In [3]:
class CustomDense(tf.keras.layers.Layer):
    def __init__(self, units, kernel_initializer, bias_initializer, spectral_normalize, **kwargs):
        super(CustomDense, self).__init__(**kwargs)
        self.units = units
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
        self.spectral_normalize = spectral_normalize
        
    
    def build(self, input_shape):
        '''
        input_shape = (batch_size, ..., input_dim)
        '''
        self.input_dim = int(input_shape[-1])
        self.kernel = self.add_weight("kernel", 
                                      shape=(self.input_dim, self.units), 
                                      initializer=self.kernel_initializer)
        self.bias = self.add_weight("bias", shape=(self.units,), initializer=self.bias_initializer)
        super(CustomDense, self).build(input_shape)  # Be sure to call this at the end
    
    
    def call(self, inputs):
        if self.spectral_normalize:
            with tf.compat.v1.variable_scope(self.name, reuse=tf.compat.v1.AUTO_REUSE):
                x = tf.matmul(inputs, spectral_norm(self.kernel))
        else:
            x = tf.matmul(inputs, self.kernel)
        x = tf.nn.bias_add(x, self.bias)
        return x

In [4]:
x = np.random.rand(2,12,14,21)
units = 11
kernel_initializer = tf.constant_initializer(2.0)
bias_initializer = tf.constant_initializer(1.5)

In [5]:
y1 = CustomDense(units, kernel_initializer, bias_initializer, True)(x)
y2 = tf.keras.layers.Dense(units, kernel_initializer=kernel_initializer, bias_initializer=bias_initializer)(x)

W1125 11:07:24.767828 10360 base_layer.py:1814] Layer custom_dense is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

W1125 11:07:24.818576 10360 base_layer.py:1814] Layer dense is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you i

In [6]:
y1

<tf.Tensor: id=66, shape=(2, 12, 14, 11), dtype=float32, numpy=
array([[[[2.1233678, 2.1233678, 2.1233678, ..., 2.1233678, 2.1233678,
          2.1233678],
         [2.1443424, 2.1443424, 2.1443424, ..., 2.1443424, 2.1443424,
          2.1443424],
         [2.2451801, 2.2451801, 2.2451801, ..., 2.2451801, 2.2451801,
          2.2451801],
         ...,
         [2.1876478, 2.1876478, 2.1876478, ..., 2.1876478, 2.1876478,
          2.1876478],
         [2.2199602, 2.2199602, 2.2199602, ..., 2.2199602, 2.2199602,
          2.2199602],
         [2.402463 , 2.402463 , 2.402463 , ..., 2.402463 , 2.402463 ,
          2.402463 ]],

        [[2.232788 , 2.232788 , 2.232788 , ..., 2.232788 , 2.232788 ,
          2.232788 ],
         [2.14844  , 2.14844  , 2.14844  , ..., 2.14844  , 2.14844  ,
          2.14844  ],
         [2.09032  , 2.09032  , 2.09032  , ..., 2.09032  , 2.09032  ,
          2.09032  ],
         ...,
         [2.1921003, 2.1921003, 2.1921003, ..., 2.1921003, 2.1921003,
        

In [7]:
y2

<tf.Tensor: id=100, shape=(2, 12, 14, 11), dtype=float32, numpy=
array([[[[20.448734, 20.448734, 20.448734, ..., 20.448734, 20.448734,
          20.448734],
         [21.086315, 21.086315, 21.086315, ..., 21.086315, 21.086315,
          21.086315],
         [24.151514, 24.151514, 24.151514, ..., 24.151514, 24.151514,
          24.151514],
         ...,
         [22.402685, 22.402685, 22.402685, ..., 22.402685, 22.402685,
          22.402685],
         [23.384893, 23.384893, 23.384893, ..., 23.384893, 23.384893,
          23.384893],
         [28.932497, 28.932497, 28.932497, ..., 28.932497, 28.932497,
          28.932497]],

        [[23.774828, 23.774828, 23.774828, ..., 23.774828, 23.774828,
          23.774828],
         [21.21087 , 21.21087 , 21.21087 , ..., 21.21087 , 21.21087 ,
          21.21087 ],
         [19.444181, 19.444181, 19.444181, ..., 19.444181, 19.444181,
          19.444181],
         ...,
         [22.538027, 22.538027, 22.538027, ..., 22.538027, 22.538027,
       